In [1]:
from config import driver, username, password, host, port, database
from sqlalchemy import create_engine, func, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import psycopg2
import pandas as pd

In [2]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

connection = engine.connect()

In [7]:
SQL_Statement = "SELECT \
                        fiscal_year, \
                        fiscal_month, \
                        SUM(tripdurationinmins) AS total_trip_duration_in_mins, \
                        COUNT(ride_id) AS trip_count, \
                        COUNT(DISTINCT trip_date) AS number_of_days_in_month, \
                        COUNT(ride_id)/COUNT(DISTINCT trip_date) AS avg_trips_per_day \
                    FROM public.citibiketripdata \
                    GROUP BY fiscal_year, fiscal_month \
                    ORDER BY fiscal_year ASC, fiscal_month ASC"
yearMonthSummaryDF = pd.read_sql(SQL_Statement,connection)
yearMonthSummaryDF

,fiscal_year,fiscal_month,total_trip_duration_in_mins,trip_count,number_of_days_in_month,avg_trips_per_day
0,2017,1,9064578.0,726676,31,23441
1,2017,2,10336651.0,791647,27,29320
2,2017,3,9217958.0,727665,28,25988
3,2017,4,23159607.0,1315404,30,43846
4,2017,5,24214924.0,1523268,31,49137
5,2017,6,30111562.0,1731594,30,57719
6,2017,7,29813961.0,1735599,31,55987
7,2017,8,34129753.0,1816498,31,58596
8,2017,9,30287852.0,1878098,30,62603
9,2017,10,32839214.0,1897592,31,61212


In [8]:
yearMonthSummaryDF.to_csv("..\Resources\yearMonthSummary.csv", index = False)

In [ ]:
SQL_Statement = "SELECT \
                        fiscal_year, \
                        fiscal_month, \
                        member_casual, \
                        COUNT(ride_id) AS trip_count \
                    FROM public.citibiketripdata \
                    GROUP BY fiscal_year, fiscal_month, member_casual \
                    ORDER BY fiscal_year ASC, fiscal_month ASC, member_casual ASC"
yearlySubscriberSummaryDF = pd.read_sql(SQL_Statement,connection)
yearlySubscriberSummaryDF

In [3]:
yearlySubscriberSummaryDF.to_csv("..\Resources\yearlySummaryBySubscribers.csv", index = False)

,fiscal_year,fiscal_month,member_casual,count
0,2017,1,casual,23018
1,2017,1,member,700465
2,2017,1,None,3193
3,2017,2,casual,42916
4,2017,2,member,741151
...,...,...,...,...
112,2021,7,member,2021205
113,2021,8,casual,1208402
114,2021,8,member,1864076
115,2021,9,casual,1390509


In [9]:
SQL_Statement = "SELECT fiscal_year, fiscal_month, trip_date, member_casual, DATE_PART('hour', started_at) AS hour_of_day, count(*) \
	FROM public.citibiketripdata \
	WHERE \
        fiscal_year IN (2020,2021) \
	GROUP BY fiscal_year, fiscal_month, trip_date, member_casual, hour_of_day \
	ORDER BY fiscal_year ASC, fiscal_month ASC, trip_date ASC, hour_of_day ASC"
    
HourlyUsageDF = pd.read_sql(SQL_Statement,connection)
HourlyUsageDF

,fiscal_year,fiscal_month,trip_date,member_casual,hour_of_day,count
0,2020,1,2020-01-01,casual,0.0,97
1,2020,1,2020-01-01,member,0.0,412
2,2020,1,2020-01-01,casual,1.0,120
3,2020,1,2020-01-01,member,1.0,513
4,2020,1,2020-01-01,casual,2.0,71
...,...,...,...,...,...,...
30398,2021,9,2021-09-30,member,21.0,2281
30399,2021,9,2021-09-30,casual,22.0,1489
30400,2021,9,2021-09-30,member,22.0,1952
30401,2021,9,2021-09-30,casual,23.0,1197


In [11]:
HourlyUsageDF.to_csv("..\Resources\HourlyUsage.csv", index = False)

In [5]:
SQL_Statement = "SELECT DISTINCT started_at::timestamp::date AS trip_date, \
                member_casual, \
                COUNT(*) as trip_count, \
                SUM(tripdurationinmins) as trip_duration_in_mins \
	FROM citibiketripdata \
	GROUP BY started_at::timestamp::date, member_casual \
	ORDER BY trip_date ASC;"
    
DaywiseUsageDF = pd.read_sql(SQL_Statement,connection)
DaywiseUsageDF

,trip_date,member_casual,trip_count,trip_duration_in_mins
0,2017-01-01,casual,3622,145800.0
1,2017-01-01,member,12387,153095.0
2,2017-01-02,casual,570,21806.0
3,2017-01-02,member,8351,80186.0
4,2017-01-03,casual,162,86147.0
...,...,...,...,...
3453,2021-09-28,member,63319,879559.0
3454,2021-09-29,casual,39759,793720.0
3455,2021-09-29,member,71773,1072917.0
3456,2021-09-30,casual,41371,791500.0


In [6]:
DaywiseUsageDF.to_csv("..\Resources\DaywiseUsage.csv", index = False)